In [10]:
import os
import pandas as pd
import datetime
import fnmatch  
# ───────────────────────────────────────────
# 설정
# INPUT_FILE   = r"C:\Users\hanta\Downloads\test.xlsx"
INPUT_FILE = r"C:\Users\hanta\Downloads\sample_kb_2040801-250711.xlsx"
MAPPING_FILE = r"C:\Users\hanta\Downloads\mapping02.xlsx"
OUTPUT_SUMM  = r"C:\Users\hanta\Downloads\monthly_summary_v04.xlsx"
# ───────────────────────────────────────────

# 1) 모든 시트 로드 (헤더 5번째 행부터 컬럼 지정)
sheets = pd.read_excel(
    INPUT_FILE,
    sheet_name=None,
    skiprows=4,
    names=[
        '거래일시','적요','보낸분/받는분','송금메모',
        '출금액','입금액','잔액','거래점','구분'
    ]
)

# 2) 매핑표 불러오기
#    usecols 로 필요한 컬럼만 읽어서 빠르게 처리
mapping_df = (
    pd.read_excel(
        MAPPING_FILE,
        dtype=str,
        usecols=['RawText','Category','입출금']
    )
    .fillna('')  # 빈값은 "" 로
)

# 2-1) (Category, 입출금) 조합 리스트 추출 (중복 제거)
combos = mapping_df[['Category','입출금']] \
           .drop_duplicates() \
           .values.tolist()

# 2-2) 텍스트 → (Category, InOut) 반환 함수
records_map = mapping_df.to_dict('records')

def map_fields_old(text: str):
    for rec in records_map:
        if rec['RawText'] and rec['RawText'] in text:
            return rec['Category'], rec['입출금']
    return '기타',''

# fnmatch 으로 와일드카드 지원하도록 map_fields 재정의
def map_fields(text: str):
    for rec in records_map:
        raw = rec['RawText']
        if not raw:
            continue
        # 와일드카드(*)가 없으면, 앞뒤에 * 붙여 부분검색처럼 동작
        pattern = raw if '*' in raw else f"*{raw}*"
        if fnmatch.fnmatch(text, pattern):
            return rec['Category'], rec['입출금']
    return '기타',''


# 3) 시트별 처리 & 레코드 수집
records = []
for period, df in sheets.items():
    df = df.dropna(subset=['거래일시']).copy()
    df['거래일시'] = pd.to_datetime(df['거래일시'], errors='coerce')

    # 금액 컬럼 숫자형으로
    df['입금액'] = pd.to_numeric(df['입금액'], errors='coerce').fillna(0)
    df['출금액'] = pd.to_numeric(df['출금액'], errors='coerce').fillna(0)

    # 보낸분/받는분 + 송금메모 → Category, InOut 컬럼
    combined = (
        df['보낸분/받는분'].fillna('').astype(str)
        + ' '
        + df['송금메모'].fillna('').astype(str)
    )
    df[['Category','InOut']] = pd.DataFrame(
        combined.map(map_fields).tolist(),
        index=df.index,
        columns=['Category','InOut']
    )

    # 3-1) 전체 합계 (변경 없음)
    total_in   = df['입금액'].sum()
    total_out  = df['출금액'].sum()
    net_income = total_in - total_out

    record = {
        'Period':            period,
        '총입금액':           total_in,
        '총출금액':           total_out,
        '순수입(입금-출금)':   net_income
    }

    # 3-2) mapping.xlsx 조합별 집계 추가
    #     Category별・입출금별로 deposit/withdraw sum
    for cat, io in combos:
        if io == '입금':
            # 입금만
            record[f'{cat}_입금'] = df.loc[
                (df['Category']==cat) & (df['InOut']=='입금'),
                '입금액'
            ].sum()
        elif io == '출금':
            # 출금만
            record[f'{cat}_출금'] = df.loc[
                (df['Category']==cat) & (df['InOut']=='출금'),
                '출금액'
            ].sum()
        else:
            # 빈값인 경우 “전체”로 → 양쪽 모두 집계
            record[f'{cat}_입금'] = df.loc[
                df['Category']==cat,
                '입금액'
            ].sum()
            record[f'{cat}_출금'] = df.loc[
                df['Category']==cat,
                '출금액'
            ].sum()

    records.append(record)

# 4) 결과 DataFrame 생성 및 엑셀 저장
summary_df = (
    pd.DataFrame(records)
      .set_index('Period')
      .sort_index()
)
summary_df.to_excel(OUTPUT_SUMM)

print(f"✅ 월별 요약을 '{OUTPUT_SUMM}'에 저장했습니다. 처리 완료: {datetime.datetime.now()}")


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\hanta\\Downloads\\sample_kb_2040801-250711.xlsx'

In [5]:
OUTPUT_UNIQUE = r"C:\Users\hanta\Downloads\unique_texts02.xlsx"

# 1) 모든 텍스트 수집
all_texts = []
for period, df in sheets.items():
    df = df.dropna(subset=['거래일시'])
    all_texts += df['보낸분/받는분'].fillna('').astype(str).tolist()
    all_texts += df['송금메모'].fillna('').astype(str).tolist()

# 2) 빈값 제거 후 Series로 변환 → value_counts() 로 개수 집계
text_series = pd.Series([t.strip() for t in all_texts if t.strip()])
counts = text_series.value_counts().rename_axis('RawText').reset_index(name='Count')

# 3) 엑셀로 저장 (RawText, Count)
counts.to_excel(OUTPUT_UNIQUE, index=False)

print(f"✅ '{OUTPUT_UNIQUE}'에 고유 텍스트 {len(counts)}개와 각 Count를 저장했습니다.")
print("   → 이 파일을 열어 RawText→Category를 채운 뒤, 'mapping.xlsx'로 저장한 다음")
print("      다시 스크립트를 실행하세요.")
sys.exit(0)


✅ 'C:\Users\hanta\Downloads\unique_texts02.xlsx'에 고유 텍스트 207개와 각 Count를 저장했습니다.
   → 이 파일을 열어 RawText→Category를 채운 뒤, 'mapping.xlsx'로 저장한 다음
      다시 스크립트를 실행하세요.


SystemExit: 0

C:\Users\hanta\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
